## Извлечение данных

In [115]:
import re
import pandas as pd
from datetime import datetime
import eval7, pprint
from poker.room.pokerstars import PokerStarsHandHistory
from poker.hand import Hand, Combo
import numpy as np
from phevaluator import evaluate_cards

In [275]:
# Создали словарь
data_pd = {'TypeOfRoom': [], 'HandId': [], 'Tournament': [], 'Level': [], 'Date': [], 'Hour': [],
           'TimeZoneAbb': [], 'Table': [], 'NoOfPlayers': [], 'bain': [], 'HandHiro': [], 'Board': []}

# Открываем файл
with open('dim1000.txt', 'r') as f:
    data = f.read()
    
# Ищем нужные данные
result = re.findall('(\w+) Hand #(\d*): Tournament #(\d*),(.*) Level (.*) - (\d{4}/\d+/\d+) (\d+\D\d+\D\d+) (\w*)', data)
Tables = re.findall('Table \'.*\' (.*) Seat .*\n', data)
bains = re.findall('(\D\d+\.\d+\+\D\d+\.\d+)', data)
HandHiros = re.findall("Dealt to dimetruk \D(.. ..)\D", data)
Boards = re.findall("Board \[(.*)\]", data)


# Считаем сколко фишек
s = re.findall('Seat (\d).*chips', data)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1
seats.append(length)


# ЗАполнЯем словарь данными
for r, table, bain, seat, handHiro, board in zip(result, Tables, bains, seats, HandHiros, Boards):
    data_pd['TypeOfRoom'].append(r[0])
    data_pd['HandId'].append(r[1])
    data_pd['Tournament'].append(r[2])
    # data_pd['info'].append(r[3])
    data_pd['Level'].append(r[4])
    data_pd['Date'].append(r[5])
    data_pd['Hour'].append(r[6])
    data_pd['TimeZoneAbb'].append(r[7])
    data_pd['Table'].append(table) #table
    data_pd['NoOfPlayers'].append(seat)
    data_pd['bain'].append(bain)
    data_pd['HandHiro'].append(handHiro)
    data_pd['Board'].append(board)

# Создаём ДатаФрейм
df = pd.DataFrame(data_pd)
df.head()

,TypeOfRoom,HandId,Tournament,Level,Date,Hour,TimeZoneAbb,Table,NoOfPlayers,bain,HandHiro,Board
0,PokerStars,226484810600,3187437334,IV (40/80),2021/05/05,19:56:10,MSK,6-max,2,$13.79+$1.21,Jh Td,8h 8c 7c 3h 6d
1,PokerStars,226484828418,3187437334,V (60/120),2021/05/05,19:56:57,MSK,6-max,2,$13.79+$1.21,Ts 3s,2s 6s As Jc Th
2,PokerStars,226484843891,3187437334,V (60/120),2021/05/05,19:57:39,MSK,6-max,2,$13.79+$1.21,4s 5c,Ks Qh 7h Tc 4c
3,PokerStars,226484859594,3187437334,V (60/120),2021/05/05,19:58:21,MSK,6-max,2,$13.79+$1.21,9c Th,7s 4d Qd
4,PokerStars,226484865470,3187437334,V (60/120),2021/05/05,19:58:37,MSK,6-max,2,$13.79+$1.21,7h 9d,2h Qd 5d Ts 3s


## Преобразуем строковые данные

In [277]:
date = data_pd['Date']
time = data_pd['Hour']

df["Date"] = pd.to_datetime(data_pd['Date'])
df["Time"] = pd.to_timedelta(data_pd['Hour'])

df["DateTime"] = df["Date"] + df["Time"]
df["DateTime"]

df.Tournament = df.Tournament.astype('int64')
df = df.astype({'HandId': np.int64, 'Tournament': np.int64})

In [290]:
# объеденяем руку и борд, выводим название комбинации

df['hand'] = df['HandHiro'] + " " + df['Board']
# hand = df['hand'].values[7].split()
# print(hand)

hand_ev = [eval7.Card(s) for s in (df['hand'].values[7].split())]
rang = eval7.evaluate(hand_ev)   # ранг комбинации
# print(rang)

eval7s = eval7.handtype(rang)  #  название комбинации

df['rang'] = eval7.evaluate(hand_ev)
df['eval7s'] =  eval7.handtype(rang)

In [292]:
df[['HandHiro', 'Board','DateTime', 'rang', 'eval7s']].head()  # применим фильтер по столбцам

,HandHiro,Board,DateTime,rang,eval7s
0,Jh Td,8h 8c 7c 3h 6d,2021-05-05 19:56:10,17392128,Pair
1,Ts 3s,2s 6s As Jc Th,2021-05-05 19:56:57,17392128,Pair
2,4s 5c,Ks Qh 7h Tc 4c,2021-05-05 19:57:39,17392128,Pair
3,9c Th,7s 4d Qd,2021-05-05 19:58:21,17392128,Pair
4,7h 9d,2h Qd 5d Ts 3s,2021-05-05 19:58:37,17392128,Pair


### df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe